In [ ]:
from pathlib import Path
import torch
from effdet import create_model, create_loader, create_evaluator
from effdet.data import resolve_input_config
try:
    from timm.layers import set_layer_config
except ImportError:
    from timm.models.layers import set_layer_config
from ml_carbucks.utils.coco import create_dataset_custom
from effdet.anchors import Anchors, AnchorLabeler

BATCH_SIZE = 16
IMG_SIZE = 320
NUM_CLASSES = None
EPOCHS = 400
extra_args = dict(image_size=(IMG_SIZE, IMG_SIZE))



In [ ]:
# TRAINING

bench_train = create_model(
    model_name='tf_efficientdet_d2', 
    bench_task='train',
    num_classes=NUM_CLASSES,
    pretrained=True,
    redundant_bias=None,
    soft_nms=None,
    checkpoint_path='',
    checkpoint_ema=False,
    **extra_args,
)
model_train_config = bench_train.config
labeler = AnchorLabeler(
    Anchors.from_config(model_train_config),
    model_train_config.num_classes,
    match_threshold=0.5,
)

train_dataset = create_dataset_custom(
    name="val",
    # img_dir=Path("/home/bachelor/ml-carbucks/data/mscoco/val2017"),
    # ann_file=Path("/home/bachelor/ml-carbucks/data/mscoco/annotations/instances_val2017.json"),
    img_dir=Path("/home/bachelor/ml-carbucks/data/car_dd/images/val"),
    ann_file=Path("/home/bachelor/ml-carbucks/data/car_dd/instances_val.json"),
)

train_input_config = resolve_input_config({}, model_train_config)
train_loader = create_loader(
    train_dataset,
    input_size=train_input_config['input_size'],
    batch_size=BATCH_SIZE,
    use_prefetcher=True,
    interpolation=train_input_config['interpolation'],
    mean=train_input_config['mean'],
    std=train_input_config['std'],
    num_workers=4,
    pin_mem=False,
    anchor_labeler=labeler,
)

for param in bench_train.model.backbone.parameters():
    param.requires_grad = False

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, bench_train.parameters()), lr=1e-4
)

bench_train = bench_train.cuda()
# set_layer_config(bench_train, 'fuse_bn', True)
bench_train.train()
for epoch in range(EPOCHS):
    for batch_idx, (input, target) in enumerate(train_loader):
        output = bench_train(input, target)
        loss = output['loss']
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.4f}")


train_state_dict = bench_train.model.state_dict()


In [ ]:
# import cloudpickle as cpkl
# cpkl.dump(train_state_dict, open("effdet_14_10_2025.pkl", "wb"))
torch.save(train_state_dict, "effdet_14_10_2025.pth")

In [ ]:
# EVALUATION

bench_pred = create_model(
    model_name='tf_efficientdet_d2', 
    bench_task='predict',
    num_classes=NUM_CLASSES,
    pretrained=True,
    redundant_bias=None,
    soft_nms=None,
    checkpoint_path='',
    checkpoint_ema=False,
    **extra_args,
)
# bench_pred.model.load_state_dict(train_state_dict)
model_pred_config = bench_pred.config
bench_pred = bench_pred.cuda()

val_dataset = create_dataset_custom(
    name="val",
    img_dir=Path("/home/bachelor/ml-carbucks/data/mscoco/val2017"),
    ann_file=Path("/home/bachelor/ml-carbucks/data/mscoco/annotations/instances_val2017.json"),
    # img_dir=Path("/home/bachelor/ml-carbucks/data/car_dd/images/val"),
    # ann_file=Path("/home/bachelor/ml-carbucks/data/car_dd/instances_val.json"),
)

input_config = resolve_input_config({}, model_pred_config)
loader = create_loader(
    val_dataset,
    input_size=input_config['input_size'],
    batch_size=BATCH_SIZE,
    use_prefetcher=True,
    interpolation=input_config['interpolation'],
    mean=input_config['mean'],
    std=input_config['std'],
    num_workers=4,
    pin_mem=False
)
evaluator = create_evaluator("coco", val_dataset, pred_yxyx=False)
bench_pred.eval()


with torch.no_grad():
    for i, (input, target) in enumerate(loader):
        output = bench_pred(input, img_info=target)
        evaluator.add_predictions(output, target)

        if i % 10 == 0:
            print(f"Eval {i}/{len(loader)}")

metrics = evaluator.evaluate()
print(metrics)

# THIS IS THE Xth ATTEMPT TO TRAIN/DEBUG IT

In [2]:
from effdet import EfficientDet, create_loader
from effdet.bench import DetBenchTrain
from effdet.config import get_efficientdet_config
from effdet.anchors import Anchors, AnchorLabeler
from effdet.data import resolve_input_config

from ml_carbucks import DATA_CAR_DD_DIR
from ml_carbucks.utils.coco import create_dataset_custom
from ml_carbucks.utils.logger import setup_logger
from ml_carbucks.utils.inference import plot_img_pred as ppp


IMG_SIZE = 320
BATCH_SIZE = 16
NUM_CLASSES = 3  # this time no background class is included into counting

logger = setup_logger("efficient_det_v2")

config = get_efficientdet_config(model_name="tf_efficientdet_d2")
setattr(config, "image_size", (IMG_SIZE, IMG_SIZE))

model = EfficientDet(config, pretrained_backbone=True)

model.reset_head(num_classes=NUM_CLASSES)

create_auto_labeler = True

bench_train = DetBenchTrain(model, create_labeler=create_auto_labeler)
model_train_config = bench_train.config

if create_auto_labeler:
    labeler = None
else:
    labeler = AnchorLabeler(
        Anchors.from_config(model_train_config),
        model_train_config.num_classes,
        match_threshold=0.25,
    )

train_dataset = create_dataset_custom(
    name="train",
    img_dir=DATA_CAR_DD_DIR / "images" / "train",
    ann_file=DATA_CAR_DD_DIR / "instances_train_curated.json",
)

train_input_config = resolve_input_config({}, model_train_config)

train_loader = create_loader(
    train_dataset,
    input_size=train_input_config["input_size"],
    batch_size=BATCH_SIZE,
    use_prefetcher=True,
    interpolation=train_input_config["interpolation"],
    mean=train_input_config["mean"],
    std=train_input_config["std"],
    num_workers=4,
    pin_mem=False,
    anchor_labeler=labeler,
)


In [ ]:
batch = next(iter(train_loader))

imgs, targets = batch
print(f"Images shape: {imgs.shape}")
ii = 3
ppp(imgs[ii], targets["bbox"][ii], coords="xyxy")

In [3]:
import torch
from tqdm import tqdm
from ml_carbucks.utils.coco import CocoStatsEvaluator
EPOCHS = 10
LR = 5e-4

train_evaluator = CocoStatsEvaluator(train_dataset, distributed=False, pred_yxyx=False)
optimizer = torch.optim.AdamW(bench_train.parameters(), lr=LR, weight_decay=1e-4)
bench_train = bench_train.cuda()

for epoch in range(EPOCHS):

    bench_train.train()
    for input, target in tqdm(train_loader):
        output = bench_train(input, target)
        loss = output["loss"]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    bench_train.eval()
    with torch.no_grad():
        for input, target in tqdm(train_loader):
            output = bench_train(input, target)
            train_evaluator.add_predictions(output['detections'], target)

    stats = train_evaluator.evaluate()
    stats = [round(s, 4) for s in stats]
    train_evaluator.reset()

    logger.info(f"map: {stats[0]}, map50: {stats[1]}")
    
        

100%|██████████| 176/176 [00:12<00:00, 14.61it/s]


INFO efficient_det_v2 19:01:26 | map: 0.0008, map50: 0.0042


100%|██████████| 176/176 [00:12<00:00, 14.57it/s]


INFO efficient_det_v2 19:02:19 | map: 0.0005, map50: 0.0028


100%|██████████| 176/176 [00:12<00:00, 14.49it/s]


INFO efficient_det_v2 19:03:12 | map: 0.0012, map50: 0.006


 36%|███▋      | 64/176 [00:12<00:21,  5.19it/s]


KeyboardInterrupt: 

# THIS is a efficient det content from the run file
It is just moved here since this works kindof like archive for future reference

In [ ]:
import datetime as dt
from typing import Any

from tqdm import tqdm
from effdet import (  # noqa F401
    create_model,
    unwrap_bench,
    create_loader,
    create_dataset,
    create_evaluator,
)
from timm.optim._optim_factory import create_optimizer_v2
from timm.scheduler.scheduler_factory import create_scheduler_v2
from timm.models.layers import set_layer_config  # type: ignore # noqa F401
from effdet.data import resolve_input_config, SkipSubset  # noqa F401
from effdet.anchors import Anchors, AnchorLabeler
import torch  # noqa F401

from ml_carbucks.patches.effdet import create_dataset_custom
from ml_carbucks.utils.logger import setup_logger
from ml_carbucks.utils.result_saver import ResultSaver  # noqa F401
from ml_carbucks import RESULTS_DIR, DATA_DIR


logger = setup_logger("effdet_v2", log_file="/home/bachelor/ml-carbucks/logs/logs.log")


class Args:
    def __init__(self, **entries):
        for key, value in entries.items():
            setattr(self, key, value)

    def __getattr__(self, name: str) -> Any:
        return self.__dict__.get(name, None)

    def vars(self):
        return self.__dict__


def create_datasets_and_loaders(
    args: Args,
    model_config: Any,
    transform_train_fn=None,
    transform_eval_fn=None,
    collate_fn=None,
):

    input_config = resolve_input_config(args, model_config)

    dataset_train = create_dataset_custom(
        img_dir=DATA_DIR / "car_dd" / "images" / "train",
        ann_file=DATA_DIR / "car_dd" / "instances_train_curated.json",
        has_labels=True,
    )

    dataset_eval = create_dataset_custom(
        img_dir=DATA_DIR / "car_dd" / "images" / "val",
        ann_file=DATA_DIR / "car_dd" / "instances_val_curated.json",
        has_labels=True,
    )

    labeler = None
    if not args.bench_labeler:
        labeler = AnchorLabeler(
            Anchors.from_config(model_config),
            model_config.num_classes,
            match_threshold=0.5,
        )

    loader_train = create_loader(
        dataset_train,
        input_size=input_config["input_size"],
        batch_size=args.batch_size,
        is_training=True,
        use_prefetcher=args.prefetcher,
        re_prob=args.reprob,
        re_mode=args.remode,
        re_count=args.recount,
        # color_jitter=args.color_jitter,
        # auto_augment=args.aa,
        interpolation=input_config["interpolation"],
        fill_color=input_config["fill_color"],
        mean=input_config["mean"],
        std=input_config["std"],
        num_workers=args.workers,
        distributed=args.distributed,
        pin_mem=args.pin_mem,
        anchor_labeler=labeler,
        transform_fn=transform_train_fn,
        collate_fn=collate_fn,
    )

    loader_eval = create_loader(
        dataset_eval,
        input_size=input_config["input_size"],
        batch_size=args.batch_size,
        is_training=False,
        use_prefetcher=args.prefetcher,
        interpolation=input_config["interpolation"],
        fill_color=input_config["fill_color"],
        mean=input_config["mean"],
        std=input_config["std"],
        num_workers=args.workers,
        distributed=args.distributed,
        pin_mem=args.pin_mem,
        anchor_labeler=labeler,
        transform_fn=transform_eval_fn,
        collate_fn=collate_fn,
    )

    evaluator = create_evaluator(
        args.dataset, loader_eval.dataset, distributed=args.distributed, pred_yxyx=False
    )

    return loader_train, loader_eval, evaluator


# NOTE: EfficientDet uses YXYX format and 0 class is reserved for background so own labels from 0..N-1 are incorrect.
def main():

    args = Args(
        model="tf_efficientdet_d0",
        pretrained_backbone=False,
        pretrained=True,
        prefetcher=True,
        device="cuda",
        amp=False,
        num_classes=3,
        opt="momentum",
        weight_decay=1e-5,
        lr=0.008,
        epochs=50,
        dataset="coco",
        root="/home/maindamian/efficientdet-pytorch/car_dd",
        batch_size=8,
        bench_labeler=False,
        distributed=False,
        pin_mem=False,
        workers=4,
        reprob=0.0,
        remode="pixel",
        recount=1,
        runtime_stamp=dt.datetime.now().strftime("%Y%m%d-%H%M%S"),
        torchscript=False,
        initial_checkpoint="",
        redundant_bias=None,
        train_interpolation="random",
    )

    bench_train = create_model(
        args.model,
        bench_task="train",
        num_classes=args.num_classes,
        pretrained=args.pretrained,
        redundant_bias=args.redundant_bias,
        label_smoothing=args.smoothing,
        legacy_focal=args.legacy_focal,
        jit_loss=args.jit_loss,
        soft_nms=args.soft_nms,
        bench_labeler=args.bench_labeler,
        checkpoint_path=args.initial_checkpoint,
    )

    bench_train_config = bench_train.config
    bench_train.cuda()
    logger.info("here")
    optimizer = create_optimizer_v2(
        bench_train,
        opt=args.opt,
        lr=args.lr,
        weight_decay=args.weight_decay,
    )

    lr_scheduler, num_epochs = create_scheduler_v2(optimizer)

    loader_train, loader_eval, evaluator = create_datasets_and_loaders(
        args, bench_train_config
    )

    # parser_max_label = loader_train.dataset.parser.max_label  # type: ignore
    # config_num_classes = bench_train_config.num_classes

    # if parser_max_label != config_num_classes:
    #     logger.error(
    #         f"Number of classes in dataset ({parser_max_label}) does not match "
    #         f"model config ({config_num_classes})."
    #     )
    #     exit(1)

    saver = ResultSaver(
        path=RESULTS_DIR / "effdet_v2",
        name=f"{args.model}_{args.runtime_stamp}",
    )

    for epoch in range(args.epochs):
        logger.info(f"Epoch {epoch + 1}/{args.epochs} starting...")

        # Training and evaluation logic would go here
        bench_train.train()

        total_loss = 0.0
        for inputs, targets in tqdm(loader_train):
            output = bench_train(inputs, targets)
            loss = output["loss"]
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step(epoch + 1)

        bench_train.eval()
        val_total_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_targets in loader_eval:
                val_output = bench_train(val_inputs, val_targets)
                val_loss = val_output["loss"]
                val_total_loss += val_loss.item()
                evaluator.add_predictions(val_output["detections"], val_targets)

                torch.cuda.synchronize()
        val_map = evaluator.evaluate().item()  # type: ignore
        evaluator.reset()
        saver.save(
            epoch=epoch + 1, loss=total_loss, val_map=val_map, val_loss=val_total_loss
        ).plot(show=False)

        logger.info(f"Epoch {epoch + 1}/{args.epochs} completed.")


if __name__ == "__main__":
    main()
